# Zero Shot Text Classification Model
**Model Description** - Bart with a classification head trained on MNLI.

Sequences are posed as NLI premises and topic labels are turned into premises, i.e. business -> This text is about business.

In [1]:
import eland as ed
from eland.conftest import *
import pandas as pd
import seaborn as sns
import preprocessor as prep
import matplotlib.pyplot as plt
import string
from tqdm.auto import tqdm
from transformers import BartForSequenceClassification, BartTokenizer

pd.set_option('display.max_colwidth', -1)

PyTorch version 1.6.0 available.
TensorFlow version 2.3.0 available.
/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


## Importing the Data from Elasticsearch

In [12]:
import os
import elasticsearch
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch()

temp_dir = '../data/'
# Add just translated data to the database (TO BE REMOVED)
df = pd.read_csv(os.path.join(temp_dir,'text_translated.csv'), lineterminator='\n')
for idx,row in df.iterrows():
    try:
        print(row['id'])
        body ={
        'doc':{
            'full_text_trans':row['text'],
            }
        }
        es.update(
            index='twitter',
            id=row['id'],
            body = body
        )
    except elasticsearch.exceptions.RequestError:
        continue

1263637941055819776
1263635326007984128
1263637939046674432
1263637900337410048
1263638449115951104
1263638342186532864
1263638090368774144
1263639381899911168
1263623731718631424
1263606442730442752
1263639246146908160
1263639209321140224
1263639048381489152
1263638574504738816
1263638541965258752
1263638482963927040
1263638457731055616
1263638891321573376
1263638817828827136
1263638725176811520
1263638613293584384
1263668733324689408
1263666053399416832
1263639139833876480
1263596418968596480
1263639379953606656
1263638951195262976
1263832409499287552
1263832362237952000
1263832338670202880
1263832330659155968
1263832563241623552
1263828530401406976
1263823489284235264
1263808576704798720
1263662372448690176
1263821677235859456
1263832539719962624
1263832436934209536
1263832417632223232
1263832415816073216
1263832457482248192
1263832438213447680
1263832437265752064
1263812354120323072
1263810905239769088
1263832304591331328
1263833164264484864
1263833162834153472
1263818532925476864


1263831194443100160
1263829562468818944
1263831139707486208
1263637421327093760
1263829107512455168
1263829094720028672
1263831280111751168
1263831270619938816
1263831266102857728
1263820387558834176
1263789786080391168
1263831067246514176
1263831053958955008
1263787781743980544
1263831321606053888
1263831315272683520
1263830994748092416
1263829460979257344
1263829164240535552
1263830983402545152
1263831350856962048
1263831346759139328
1263831342170734592
1263831324873363456
1263830464151814144
1263831028826701824
1263060067831689216
1263045506407809024
1263054674753552384
1263047936381841408
1263047135177523200
1263060345834397696
1263060297734127616
1263060292029857792
1263060283951480832
1263049845838737408
1263049973479796736
1263060101994131456
1263060101914660864
1263059911866413056
1263059737211367424
1263060135221608448
1263060121501876224
1263049250402787328
1263049718252199936
1263060719899213824
1263060718816882688
1263060715247505408
1263060282001088512
1263060275328155648


1263572197957701632
1263832338338906112
1263833406972080128
1263810188261367808
1263833395366227968
1263833214180904960
1263833194362736640
1263833169662562304
1263833168106250240
1263833670542143488
1263832847820955648
1263833660253302784
1263833871705083904
1263833824942714880
1263833802511548416
1263833636748455936
1263811753084780544
1263833636215947264
1263051934799466496
1263051932328955904
1263051916671623168
1263051852045676544
1263052088134602752
1263052086847078400
1263052061727289344
1263052104672870400
1263052102764560384
1263039267531558912
1263052096074547200
1263052119512240128
1263052113808175104
1263052112281264128
1263052209165484032
1262982593957384192
1263052208406441984
1263052145764511744
1263052007100813312
1263051995604307968
1263051977925079040
1263052051837259776
1263052050104975360
1263052021281591296
1263052139502473216
1263044660013957120
1263052132434968576
1263052126449737728
1263051962016096256
1263820008745922560
1263827660007825408
1263827589497589760


1263061987405283328
1263061942987620352
1263047824780001280
1263037757305303040
1263050844687290368
1263832214589956096
1263832213327458304
1263060754917294080
1263059952056193024
1263060742141472768
1263058282739937280
1263051030226300928
1263061679132327936
1263055667365064704
1263061678276538368
1263061670991020032
1263047899530715136
1263034454286544896
1263046028770471936
1263062493556903936
1263062486632165376
1263821927195369472
1263819864600272896
1263679109005279232
1263823768901713920
1263823748534136832
1263815156863127552
1263822216459771904
1263822147513839616
1263822109647593472
1263822097924665344
1263822210202025984
1263822235048964096
1263822220792492032
1263819180836438016
1263817677715369984
1263823726593708032
1263817124021231616
1263823689591795712
1263823677872840704
1263823371436949504
1263823340487278592
1263815980767158272
1263823330026631168
1263823280210948096
1263823445143556096
1263823442970722304
1263823432635920384
1263823389837287424
1263822095512985600


1263062938400800768
1263049779090776064
1263062927948394496
1263041720121344000
1263047082815959040
1263063050149478400
1263063041018482688
1263032932286988288
1263063059595231232
1263661196923805696
1263671536361373696
1263671488433053696
1263671445307129856
1263667124653953024
1263624230312333312
1263663920230748160
1263663886286442496
1263663814299566080
1263583669685370880
1263573593893539840
1263671634306744320
1263671598004883456
1263671374968750080
1263671351400939520
1263671349081505792
1263671336963981312
1263671311122837504
1263671255091130368
1263671229925523456
1263668931648262144
1263611132117504000
1263603294401953792
1263601875645014016
1263601618697818112
1263586880311812096
1263671706922569728
1263668943417495552
1263671638450716672
1263664969612234752
1263632157806940160
1263631611901415424
1263631350311067648
1263620101133209600
1263612484218486784
1263612186322223104
1263668708733566976
1263667302417006592
1263433454378721280
1263475680513339392
1263334462877691904


...

1263471511899459584
1263434999677812736
1263470395790950400
1263459921439232000
1263471105127395328
1263476939165179904
1263457673007247360
1263468637953851392
1263477348973805568
1263477346461507584
1263477316442894336
1263477315612356608
1263456394113662976
1263451710065795072
1263447621521342464
1263442244817780736
1263477496143626240
1263477483850080256
1263477466330431488
1263477173509410816
1263477159894691840
1263477148934963200
1263477146514849792
1263477146317709312
1263477141079023616
1263477127439155200
1263477110292824064
1263477108661260288
1263477104995438592
1263475476473012224
1263475869869375488
1263477092198625280
1263477021893672960
1263473006233485312
1263451016223240192
1263454754903961600
1263448399581495296
1263438124874530816
1263482848205471744
1263474548927799296
1262426218151976960
1263470501814505472
1263364594283606016
1263421358412726272
1263476959914442752
1263332788016562176
1263425114659414016
1263471536717074432
1263457515095781376
1263470104932683776


1263350088446795776
1263350084210577408
1263350078900629504
1263095105176600576
1263138926564225024
1263131288225157120
1263085023961288704
1263332930199289856
1263332897844367360
1263332832098701312
1263332817817047040
1263209750122115072
1263334306937737216
1263334287132286976
1263334150511169536
1263334136552607744
1263521161499029504
1263521136291258368
1263521116527775744
1263332970468802560
1263332948738097152
1263332970053566464
1263332944472653824
1263521095778308096
1263508114948702208
1263508076335751168
1263508049920249856
1263508041602908160
1263508036989181952
1263507954810191872
1263508034267078656
1263508021428334592
1263316110784380928
1263521298648465408
1263521297419718656
1263521274053169152
1263521255283703808
1263521231942365184
1263521209829949440
1263505138653650944
1263507980474966016
1263507850241822720
1263507961139228672
1263504839377649664
1263504801079451648
1263504833119694848
1263504808016842752
1263504782041460736
1263494253583282176
1263493478916296704


1263370307823456256
1263369739516207104
1263369735061807104
1263369729185587200
1263369718272012288
1263367067778121728
1263363918870831104
1263362678623535104
1263369753403568128
1263369744633065472
1263369801004544000
1263368004915433472
1263369789071699968
1263369783199834112
1263368226450272256
1263366445632806912
1263351167511408640
1263336526848589824
1263336526815125504
1263336113944637440
1263369951328387072
1263369945733349376
1263369315358838784
1263368728059789312
1263368591036026880
1263369769232871424
1263367897130438656
1263365362869026816
1263364611996291072
1263350696746708992
1263350684067356672
1263350623459819520
1263350612663644160
1263350539745722368
1263349702688292864
1263341608403111936
1263349697772580864
1263350910442336256
1263350809355595776
1263350803965923328
1263350796319686656
1263350776988078080
1263350771539533824
1263350882541936640
1263350834068246528
1263350824199053312
1263352524066742272
1263352488524017664
1263352459298091008
1263352456919957504


1263362247163666432
1263362243112116224
1263362428701728768
1263362426717667328
1263362344308146176
1263362343335088128
1263362065269428224
1263362463296294912
1263362445885747200
1263335676482732032
1263335661362458624
1263335571465830400
1263335470341042176
1263334293469716480
1263335457984638976
1263329153253732352
1263335858800861184
1263335856208740352
1263335819286204416
1263335794380546048
1263335781378252800
1263352874660253696
1263352847992786944
1263352815843450880
1263351534672322560
1263351516955467776
1263351474425298944
1263351453764079616
1263351417965735936
1263351395597475840
1263351387733159936
1263353089366556672
1263353067430264832
1263353042948231168
1263352947993346048
1263351615131697152
1263351596932460544
1263351585322590208
1263351564649013248
1263352921833553920
1263330210147074048
1263352916716511232
1263352908126408704
1263352893417099264
1263335456323915776
1263335450053312512
1263335449365528576
1263335776735031296
1263335721231933440
1263335686691786752


1263504463354122240
1263504302057742336
1263424554338140160
1263494178559803392
1263494447276257280
1263494442947878912
1263494388904333312
1263492878040305664
1263496252928466944
1263496242107240448
1263496218719780864
1263496215200624640
1263496214781349888
1263494512027897856
1263494508391604224
1263494501936574464
1263494500061716480
1263494476565229568
1263496130425425920
1263496104273801216
1263496100868128768
1263496077673664512
1263495932374659072
1263496212738605056
1263495770352881664
1263494411263991808
1263494459951509504
1263496328195092480
1263496287338541056
1263496266220216320
1263489864412127232
1263489087450710016
1263494369484603392
1263496157042561024
1263496156497182720
1263342649261596672
1263369128129081344
1263369126556438528
1263336440597032960
1263369119761526784
1263146789806370816
1263794234907226112
1263732901004402688
1263859022333046784
1263369267350667264
1263369265840746496
1263362595110531072
1263369258156916736
1263369256537919488
1263369216176132096


In [31]:
ed_df = ed.DataFrame('localhost', 'twitter', columns=['full_text_trans'])

# defining the full-text query we need: Retrieving records for full_text_processed with the condition is_retweet=False and is_quote_status=False
query_unique = {
    "bool": {
        "must": {
            "term":{"is_retweet":"false"},
        },
        "filter": {
            "term":{"is_quote_status":"false"}
        },
    }
}
# using full-text search capabilities with Eland:
df_ed = ed_df.es_query(query_unique)
df_tweets = df_ed.to_pandas()
df_tweets = df_tweets.dropna()

In [32]:
df_tweets.shape

(108238, 1)

In [33]:
df_tweets.head(50)

,full_text_trans
1263748200843927552,pm modi ji announce 1000 cr interim assistance west bengal jai hind 🙏 amphan supercycloneamphan westandtogether westbengal bjym mamataofficial pmoindia
1263748254447083520,prime minister narendra modi announces r 1000 crore aid state cope damage caused cyclone amfan
1263745588849291264,padmajajoshi nistula mamwhy r u telling odishaodisha ha borne longest period havoc created amphanagain corona also present hereplease discriminate
1263748269420806144,pm modi holding review meeting chief minister mamata banerjee governor jagdeep dhankhare official basirhat west bengal regarding amfan cyclone cycloneamphan
1263748274059706368,cmoodisha otvnews gyanardas dmbhadrak energyodisha nescobalasore dark amphan till today 3 day passed 21st centurydo work ground sir fm bachhada jagannathpur basudevpur bhadrak
...,...
1263042199123636224,sagarikanath sarangisudhansu cpbbsrctc amphan blow away corona amphansupercyclone amphan
1263042168962404352,cycloneamphan infant becomes cyclone amphan’s first victim barrel odisha
1263042647108640768,get ready power bank emergency light candle etc possibility power safe amphan
1263042616272146432,cyclone amphan bear million people


## Basic Tweet Preprocessing
- Remove URLs and reserved words (RTs)
- Remove # and @ symbols
- Remove tweets less than 4 tokens in length


In [34]:
## Set options for the tweet-preprocessor
prep.set_options(prep.OPT.URL, prep.OPT.RESERVED, prep.OPT.EMOJI, prep.OPT.SMILEY)

## Clean text and remove #,@ symbols
def clean_tweet(text):
    text = prep.clean(text)
    table = str.maketrans('','','#@')
    return text.translate(table)

In [35]:
df_tweets['full_text_processed'] = df_tweets['full_text_trans'].apply(lambda x: clean_tweet(x))

In [36]:
df_tweets['length'] = df_tweets['full_text_processed'].apply(lambda x: len([w for w in x.split()]))
#df_tweets = df_tweets[df_tweets['length']>4]

In [37]:
df_tweets.head()

,full_text_trans,full_text_processed,length
1263748200843927552,pm modi ji announce 1000 cr interim assistance west bengal jai hind 🙏 amphan supercycloneamphan westandtogether westbengal bjym mamataofficial pmoindia,pm modi ji announce 1000 cr interim assistance west bengal jai hind amphan supercycloneamphan westandtogether westbengal bjym mamataofficial pmoindia,19
1263748254447083520,prime minister narendra modi announces r 1000 crore aid state cope damage caused cyclone amfan,prime minister narendra modi announces r 1000 crore aid state cope damage caused cyclone amfan,15
1263745588849291264,padmajajoshi nistula mamwhy r u telling odishaodisha ha borne longest period havoc created amphanagain corona also present hereplease discriminate,padmajajoshi nistula mamwhy r u telling odishaodisha ha borne longest period havoc created amphanagain corona also present hereplease discriminate,19
1263748269420806144,pm modi holding review meeting chief minister mamata banerjee governor jagdeep dhankhare official basirhat west bengal regarding amfan cyclone cycloneamphan,pm modi holding review meeting chief minister mamata banerjee governor jagdeep dhankhare official basirhat west bengal regarding amfan cyclone cycloneamphan,20
1263748274059706368,cmoodisha otvnews gyanardas dmbhadrak energyodisha nescobalasore dark amphan till today 3 day passed 21st centurydo work ground sir fm bachhada jagannathpur basudevpur bhadrak,cmoodisha otvnews gyanardas dmbhadrak energyodisha nescobalasore dark amphan till today 3 day passed 21st centurydo work ground sir fm bachhada jagannathpur basudevpur bhadrak,23


In [47]:
df_tweets = df_tweets[df_tweets.length != 0]

## GPU approach using Transformers Pipeline

In [48]:
from transformers import pipeline

In [49]:
classifier = pipeline('zero-shot-classification', device=0)

loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large-mnli/config.json from cache at /home/ubuntu/.cache/torch/transformers/a35b79dc26c2f371a0e19eae44d91c0a0281a5db09044517d2675703791ee3c5.746d7ef19ade685cd3ee03f131a96fab513947c26179546289ddf02a6ac683ce
Model config BartConfig {
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "id2label": {
    "0": "contradiction",
    "1": "neutral",
    "2": "entailment"
  },
  "init_std":

In [50]:
TERMS = list({'sympathy', 'criticism', 'hope', 'job', 'relief measures', 'compensation', 'evacuation', 'ecosystem', 'government', 'corruption', 'news updates', 'volunteers', 'donation', 'cellular network', 'housing', 'farm', 'utilities', 'water supply', 'power supply', 'food supply', 'medical assistance', 'coronavirus', 'petition', 'poverty', 'assistance required'})

In [51]:
%timeit classifier(df_tweets['full_text_processed'][0], TERMS, multi_class=True)

98.8 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [52]:
'''Method to get the labels for a tweet based on threshold specified'''
def get_all_labels(x, terms=TERMS):
    
    # Run model
    result = classifier(x, terms, multi_class=True)
    
    topics = []
    for label, score in zip(result['labels'], result['scores']):

        topics.append((label, np.round(score,2)))
            
    return topics

In [ ]:
tqdm.pandas()
df_tweets['full_text_processed'].dropna().progress_apply(lambda x: get_all_labels(x, TERMS)).to_json('../models/zstc_labels.json', orient='index')

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed